# LangChain QA

All code comes from [LangChain docs](langchain.readthedocs.io).

In [1]:
!pip install langchain openai chromadb tiktoken pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 9.3 MB/s eta 0:00:00


In [1]:
import os 
os.environ["OPENAI_API_BASE"] = "https://openai.api2d.net/v1"
os.environ["OPENAI_API_KEY"] = "fk193574-JC4P6uf3KOUAd3Xipa0wI8XU5HdpyDIg"

In [2]:
# @Author  : luoyiwen
# @File    : guandan_qa.py
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.prompts import HumanMessagePromptTemplate, PromptTemplate, ChatPromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA, LLMChain, SimpleSequentialChain
import os


In [3]:
# 加载文件夹中的所有txt类型的文件
loader = PyPDFLoader('./file/掼蛋App-2.6.6.pdf')
# 将数据转成 document 对象，每个文件会作为一个 document
pdf = loader.load()

# 初始化加载器
text_splitter = CharacterTextSplitter(chunk_size=20, chunk_overlap=0)
# 切割加载的 document
split_docs = text_splitter.split_documents(pdf)

# 初始化 openai 的 embeddings 对象
embeddings = OpenAIEmbeddings()
# 将 document 通过 openai 的 embeddings 对象计算 embedding 向量信息并临时存入 Chroma 向量数据库，用于后续匹配查询
docsearch = Chroma.from_documents(split_docs, embeddings)

# 创建问答对象
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model='gpt-3.5-turbo',temperature=0), chain_type="stuff", retriever=docsearch.as_retriever(), return_source_documents=False)
# 进行问答

In [4]:

# 产品描述：需求背景-每日任务/活动
result = qa({"query":
'''
新增了几个产品需求？分别总结和抽取每个需求的需求背景描述、产品界面变化和用户交互流程，对与每个产品需求请使用如下的JSON格式返回数据
{{
  "产品需求":"a",
  "需求背景描述":"b",
  "产品界面变化":"c",
  "用户交互流程":"d"
}}
Extracted:
'''
})
print(result['result'])

# todo: CHECK 1



{
  "产品需求": "升级挑战",
  "需求背景描述": "提高游戏局数和留存",
  "产品界面变化": "每周可获得一张级牌，通过完成每日任务将级数升级，升级数1-3级，完成对应任务后可领取奖励。",
  "用户交互流程": "玩家每周获得一张级牌，完成每日任务升级级数，领取奖励。"
}

{
  "产品需求": "简洁版动画",
  "需求背景描述": "去掉复杂的出牌动画效果，仅显示牌型名称",
  "产品界面变化": "在对局设置中，出牌动画效果只显示牌型名称",
  "用户交互流程": "在对局设置中选择出牌动画效果，仅显示牌型名称"
}

{
  "产品需求": "每日任务增加修改昵称任务",
  "需求背景描述": "增加每日任务的多样性，提供修改昵称的任务",
  "产品界面变化": "每日任务中增加修改昵称任务，完成后任务栏消失",
  "用户交互流程": "在每日任务中完成修改昵称任务，获得1000游戏豆奖励"
}


In [5]:
# 输出每个步骤对应的具体测试

# task1: 获取测试描述，对应用户输入的通用限制和界面展示的通用限制
_input = qa({"query": f"结合上面的产品需求，{result['result']}，输出产品需求需要哪几个输入？"})
_restrict = qa({"query": f"结合上面的产品需求，{result['result']}，输出对应的输入有哪几个限制？"})
_task = qa({"query": f"结合上面的产品需求，{result['result']}，输出当前任务的测试项"})
_pre = qa({"query": f"结合上面的产品需求，{result['result']}，输出当前任务的预置条件"})
_steps = qa({"query": f"结合上面的产品需求，{result['result']}，输出当前任务的测试步骤"})
query_input1 = [_input, _restrict, _task, _steps,_pre]
print(query_input1)

[{'query': '结合上面的产品需求，{\n  "产品需求": "升级挑战",\n  "需求背景描述": "提高游戏局数和留存",\n  "产品界面变化": "每周可获得一张级牌，通过完成每日任务将级数升级，升级数1-3级，完成对应任务后可领取奖励。",\n  "用户交互流程": "玩家每周获得一张级牌，完成每日任务升级级数，领取奖励。"\n}\n\n{\n  "产品需求": "简洁版动画",\n  "需求背景描述": "去掉复杂的出牌动画效果，仅显示牌型名称",\n  "产品界面变化": "在对局设置中，出牌动画效果只显示牌型名称",\n  "用户交互流程": "在对局设置中选择出牌动画效果，仅显示牌型名称"\n}\n\n{\n  "产品需求": "每日任务增加修改昵称任务",\n  "需求背景描述": "增加每日任务的多样性，提供修改昵称的任务",\n  "产品界面变化": "每日任务中增加修改昵称任务，完成后任务栏消失",\n  "用户交互流程": "在每日任务中完成修改昵称任务，获得1000游戏豆奖励"\n}，输出产品需求需要哪几个输入？', 'result': '输出产品需求需要以下几个输入：\n1. 产品需求：描述产品需求的名称或标题。\n2. 需求背景描述：解释产品需求的目的和原因。\n3. 产品界面变化：说明产品界面将如何改变或更新以满足需求。\n4. 用户交互流程：描述用户在产品界面上如何与需求进行交互的流程。'}, {'query': '结合上面的产品需求，{\n  "产品需求": "升级挑战",\n  "需求背景描述": "提高游戏局数和留存",\n  "产品界面变化": "每周可获得一张级牌，通过完成每日任务将级数升级，升级数1-3级，完成对应任务后可领取奖励。",\n  "用户交互流程": "玩家每周获得一张级牌，完成每日任务升级级数，领取奖励。"\n}\n\n{\n  "产品需求": "简洁版动画",\n  "需求背景描述": "去掉复杂的出牌动画效果，仅显示牌型名称",\n  "产品界面变化": "在对局设置中，出牌动画效果只显示牌型名称",\n  "用户交互流程": "在对局设置中选择出牌动画效果，仅显示牌型名称"\n}\n\n{\n  "产品需求": "每日任务增加修改昵称任务",\n  "需求背景描述": "增加每日任务的多

In [6]:
# task1: 获取测试描述，对应用户输入的通用限制和界面展示的通用限制
_input = qa({"query": "产品需求需要哪几个输入？"})
_restrict = qa({"query": "对应的输入有哪几个限制？"})
_task = qa({"query": "当前任务的测试项"})
_pre = qa({"query": "当前任务的预置条件"})
_steps = qa({"query": "当前任务的测试步骤"})
query_input2 = [_input, _restrict, _task, _steps,_pre]
print(query_input2)

[{'query': '产品需求需要哪几个输入？', 'result': '根据提供的上下文，产品需求需要以下几个输入：\n\n1. 需求目的：明确产品需求的目标和目的。\n2. 需求方案：提供解决方案来满足需求目的。\n3. 任务配置：确定每日任务的类型和配置，包括任务类型和任务奖励等。\n4. 组队界面banner优化：配置组队界面的banner链接和点击banner吊起的购买界面。\n5. 增加互动表情礼包：配置互动表情礼包的购买入口、礼包配置和界面相关内容。\n6. 其他小优化：包括王者归来弹窗优化、更换loading页、简洁版动画和每日任务增加修改昵称任务的具体要求和配置。\n\n请注意，以上是根据提供的上下文推测的需求输入，具体的需求输入可能还需要根据实际情况进行补充。'}, {'query': '对应的输入有哪几个限制？', 'result': '根据提供的上下文，没有提到任何与输入相关的限制。因此，无法确定对应的输入限制是什么。'}, {'query': '当前任务的测试项', 'result': '根据提供的上下文，没有提到当前任务的具体测试项。因此，无法提供当前任务的测试项。'}, {'query': '当前任务的测试步骤', 'result': '抱歉，根据提供的上下文，没有提到当前任务的具体内容和测试步骤。请提供更多相关信息，以便我能够帮助您回答问题。'}, {'query': '当前任务的预置条件', 'result': '根据提供的上下文，当前任务的预置条件可能包括以下内容：\n\n- 每个账号只能完成一次修改昵称任务。\n- 完成每日任务可以获得升级点数，但仅限于游戏豆场和掼神币场。\n- 终极大礼的奖励每周只能领取一次，已完成升级但未领取的终极大礼奖励将在每周的“升级大挑战”重置后通过邮件补偿发放。\n- 首充礼包和30万概率礼包的具体配置和链接需要等待美术出图后生成。\n- 互动表情礼包可以在商城的道具分类下购买。\n\n请注意，这些条件是根据上下文推测的，具体的预置条件可能还有其他内容。'}]


In [7]:
human_msg_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=
        '''
        根据当前每个产品需求对应的{query_input}JSON数据、测试项，总结和抽取相应的测试描述，按以下格式返回：
        {{
          "产品需求":"a",
          "需要的用户输入":"b",
          "用户输入的通用限制":"c",
          "界面展示的通用限制":"d",
          "当前任务的测试步骤":"e",
        }}       
        Extracted: 
        ''',
        input_variables=["query_input"]
    )
)
chat_prompt_template = ChatPromptTemplate.from_messages([human_msg_prompt])
chain = LLMChain(llm=ChatOpenAI(model='gpt-3.5-turbo',temperature=0), prompt=chat_prompt_template)

In [8]:
# task2：生成对应步骤的测试文案
second_prompt = PromptTemplate(
        template=
        '''
        测试项是指产品需求中一个功能点需要测试内容的总结(例如，升级大挑战弹窗UI展示)，预置条件是指当执行测试时，需要提前准备的工作(例如，首次点击, 也可为无); 期望测试结果为对应测试步骤的预期的测试结果(例如,升级大挑战弹窗关闭;)。给定对应的JSON数据，根据%测试描述 {test_json}中"当前任务的测试步骤"字段，对每一个字段分别生成具体的测试步骤，按以下格式返回：
        {{
        "测试项":"a",
        "预置条件":"b",
        "测试步骤":"c",
        "测试期望结果":"d",
        }}       
        Extracted: 
        ''',
        input_variables=["test_json"]
)
##to do 把对应的测试结果逻辑对应到测试步骤里面
chain_two = LLMChain(llm=ChatOpenAI(model='gpt-3.5-turbo',temperature=0.9), prompt=second_prompt)

overall_chain = SimpleSequentialChain(chains=[chain, chain_two])


# 将 query_input 数据作为输入传递给第一个 chain
output_from_chain1 = chain.run(query_input1)

# todo: CHECK 2

# 将输出结果作为输入传递给第二个 chain
output_from_chain2 = overall_chain.run(output_from_chain1)

# todo: CHECK 3

# 输出第二个 chain 的结果
print(output_from_chain2)

[
  {
    "测试项": "打开应用并登录账号",
    "预置条件": "无",
    "测试步骤": "打开应用并登录账号。",
    "测试期望结果": "成功打开应用并登录账号。"
  },
  {
    "测试项": "进入对局设置界面",
    "预置条件": "无",
    "测试步骤": "进入对局设置界面。",
    "测试期望结果": "成功进入对局设置界面。"
  },
  {
    "测试项": "确认是否存在出牌动画效果选择项",
    "预置条件": "无",
    "测试步骤": "确认是否存在出牌动画效果选择项。",
    "测试期望结果": "存在出牌动画效果选择项。"
  },
  {
    "测试项": "如果存在，选择简洁版动画",
    "预置条件": "出牌动画效果选择项存在",
    "测试步骤": "选择简洁版动画。",
    "测试期望结果": "成功选择简洁版动画。"
  },
  {
    "测试项": "进入每日任务界面",
    "预置条件": "无",
    "测试步骤": "进入每日任务界面。",
    "测试期望结果": "成功进入每日任务界面。"
  },
  {
    "测试项": "检查是否存在修改昵称任务",
    "预置条件": "无",
    "测试步骤": "检查是否存在修改昵称任务。",
    "测试期望结果": "存在修改昵称任务。"
  },
  {
    "测试项": "如果存在，完成修改昵称任务",
    "预置条件": "修改昵称任务存在",
    "测试步骤": "完成修改昵称任务。",
    "测试期望结果": "成功完成修改昵称任务。"
  },
  {
    "测试项": "确认任务栏中修改昵称任务是否消失",
    "预置条件": "修改昵称任务已完成",
    "测试步骤": "确认任务栏中修改昵称任务是否消失。",
    "测试期望结果": "修改昵称任务已消失。"
  },
  {
    "测试项": "领取修改昵称任务的奖励，检查是否获得1000游戏豆",
    "预置条件": "修改昵称任务已完成",
    "测试步骤": "领取修改昵称任务的奖励，检查是否获得1000游戏豆。",
